# Yelp API Calls (Core)

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time

from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## Snippets

In [2]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it does exist:
    if file_exists == True:
        
        ## Check if user wants to delete it if it exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## Delete file and confirm it no longer exits
            os.remove(JSON_FILE)
            
            ## Recursive call to function after old file is deleted
            create_json_file(JSON_FILE, delete_if_exists=False)

        else:
            print(f"[i] {JSON_FILE} already exists.")            

    ## If it does not exist:
    else:
        
        ## Inform user and save empty list
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## Create required folders
        ## Get the folder name
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder) > 0:
            
            # Create the folder
            os.makedirs(folder, exist_ok = True)
        
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            
            json.dump([],f)  


## Code

In [3]:
## Load API credentials
with open('/Users/ShPatel/.secret/yelp_api.json') as f:
    login = json.load(f)

## Instantiate YelpAPI variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
## Set API call parameters

location = 'NY,NY'
term = 'Pizza'

In [6]:
# Specify the JSON_FILE filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [7]:
## Create a new empty json file (delete the previous one if it exists)
create_json_file(JSON_FILE, delete_if_exists = True)

## Load previous results and use length of the results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

## Use our YelpAPI variable's search_query method to perform our API call
results = yelp_api.search_query(location = location, term = term, offset = n_results)

## How many results are there in total?
total_results = results['total']

## How many businesses did we get the details for?
results_per_page = len(results['businesses'])

## Use math.ceil to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


620

In [8]:
for i in tqdm_notebook(range(1, n_pages + 1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    
    ## Save the number of results to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## Use n_results as the offset 
    results = yelp_api.search_query(location = location, term = term, offset = n_results)
    
    ## Append new results and save to file
    previous_results.extend(results['businesses'])
    
    # Display previous results
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/620 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [9]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2708,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5087,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3195,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/1-VBwA...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,1151,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
4,uc5qQMzs96rzjK27epDCug,joes-pizza-new-york-4,Joe's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/FKjd9R...,False,https://www.yelp.com/biz/joes-pizza-new-york-4...,3072,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.73060076, 'longitude': -74.002...",[delivery],$,"{'address1': '7 Carmine St', 'address2': '', '...",+12123661182,(212) 366-1182,2874.702514


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,DF7PV0DE9z0LM4ae6XYeYg,harrys-italian-new-york-2,Harry's Italian,https://s3-media1.fl.yelpcdn.com/bphoto/lEWDOb...,False,https://www.yelp.com/biz/harrys-italian-new-yo...,648,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.707555318913, 'longitude': -74...","[delivery, pickup]",$$,"{'address1': '2 Gold St', 'address2': '', 'add...",+12127470797,(212) 747-0797,1088.108650
996,DI_FoHY84hfzTWwlzF76aw,benito-one-new-york,Benito One,https://s3-media2.fl.yelpcdn.com/bphoto/C9WAAh...,False,https://www.yelp.com/biz/benito-one-new-york?a...,828,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 40.7200188460948, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '174 Mulberry St', 'address2': ''...",+12122269171,(212) 226-9171,1635.847620
997,CqIBdZe-9rrkfKjzzynKVQ,essex-pizzeria-lodi,Essex Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/pfSMyN...,False,https://www.yelp.com/biz/essex-pizzeria-lodi?a...,19,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.8947, 'longitude': -74.07765}","[delivery, pickup]",NaN,"{'address1': '22 Essex St', 'address2': None, ...",+12014149311,(201) 414-9311,22173.754229
998,ID5DkaAOMpTmkAFR8bVLLQ,lost-in-paradise-rooftop-long-island-city,Lost in Paradise Rooftop,https://s3-media2.fl.yelpcdn.com/bphoto/7Ms1yS...,False,https://www.yelp.com/biz/lost-in-paradise-roof...,244,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.752088, 'longitude': -73.947387}",[],NaN,"{'address1': '11-01 43rd Ave', 'address2': Non...",+19297893600,(929) 789-3600,6523.243695
999,8S20qj7xeM-kxDZIrF3EvA,rustico-taverna-queens,Rustico Taverna,https://s3-media2.fl.yelpcdn.com/bphoto/9wY-Ar...,False,https://www.yelp.com/biz/rustico-taverna-queen...,71,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.70925, 'longitude': -73.89827}","[delivery, pickup]",NaN,"{'address1': '65-17 Fresh Pond Rd', 'address2'...",+13477992880,(347) 799-2880,8097.442021


In [12]:
## Check for duplicate results
#final_df.duplicated().sum()

## Check for duplicate ID's
final_df.duplicated(subset = 'id').sum()

108

In [13]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset = 'id')
final_df.duplicated(subset = 'id').sum()

0

In [15]:
# Save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression = 'gzip', index = False)